# TelcoX Churn Analysis

TelecoX  is  a  national  telecommunication  company  that  is  facing  an  increasing  churning 
problem due to the privatization of the market. 
 
In order to mitigate this issue, the head of the marketing department is aiming to offer a 20% 
discount for customers who are likely to churn. Hence it important to predict accurately the 
propensity of churn of each customer while reducing the number of false positive. 